<a href="https://colab.research.google.com/github/haeuni7/Test_git/blob/main/baseline_shopping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 쇼핑 고객 상담 대화 문장 의도 태깅

- 2차 모의경진대회(22.11.28 ~ 22.12.25)
- Text Classification 과제

## 디렉터리 구조
```
$ Shopping/
├── DATA/
│    ├── train/
│    │    ├── texts/
│    │    │    ├── shopping1_0001.txt
│    │    │    ├── ...
│    │    │    └── shopping7_2536.txt
│    │    └── labels/
│    │          ├── shopping1_0001.json
│    │          ├── ...
│    │          └── shopping7_2536.json
│    ├── test/
│    │    └── texts/
│    │          ├── test_0001.txt
│    │          ├── ...
│    │          └── test_1456.txt
│    └── sample_submission.csv
│
└── baseline/
      ├── config/
      │    ├── train.yaml
      │    └── predict.yaml
      │── modules/
      │    ├── earlystopper.py
      │    ├── losses.py
      │    ├── recorders.py
      │    └── utils.py
      ├── results/
      │    └── train/
      ├── wandb/ # wandb 최초 사용 후 생성
      ├── baseline_shopping.ipynb
      └── submission.csv (코드 실행 후 생성되는 추론 파일)

```
    - config : 학습/추론에 필요한 파라미터 등을 기록하는 yaml 파일
    - modules
        - earlystopper.py : loss가 특정 에폭 이상 개선되지 않을 경우 학습을 멈춤
        - losses.py : config에서 지정한 loss function을 리턴
        - recorders.py : log, learning curve, best model.pt 등을 기록
        - utils.py : 여러 확장자 파일을 불러오거나 여러 확장자로 저장하는 등의 함수
    - results
      - train/ : 학습 log, 추론 log를 기록하는 디렉토리
    - baseline_shopping.ipynb : 전처리부터 학습, 추론까지 수행할 코드 

## 노트북 실행 요령
- 해당 베이스라인 코드는 [Wandb](https://wandb.ai/site) 를 활용한 실험 관리가 가능합니다.
- 해당 베이스라인 코드 실행 전 `config/train.yaml`을 수정하세요.
- 해당 베이스라인 코드의 추론 실행 전 `config/predict.yaml`을 수정하세요.
- 학습 결과는 `results/train/train_serial` 에 저장되며, wandb를 통해 학습 그래프를 확인하실 수 있습니다.

#0. 사전 준비

### 구글 드라이브 마운트

In [ ]:
# 구글 Colaboratory 를 사용하기 위해 구글 계정으로 로그인합니다. 
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/MyDrive/Shopping/')

### 라이브러리 설치

In [ ]:
!pip install transformers
!pip install evaluate
!pip install omegaconf
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 4.6 MB/s 
     |████████████████████████████████| 7.6 MB 38.2 MB/s 
     |████████████████████████████████| 182 kB 75.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 72 kB 1.4 MB/s 
     |████████████████████████████████| 132 kB 8.6 MB/s 
     |████████████████████████████████| 451 kB 44.9 MB/s 
     |████████████████████████████████| 212 kB 62.4 MB/s 
     |████████████████████████████████| 127 kB 59.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 79 kB 3.7 MB/s 
     |██████████████████████

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 4.6 MB/s 
     |████████████████████████████████| 182 kB 73.1 MB/s 
     |████████████████████████████████| 168 kB 56.3 MB/s 
     |████████████████████████████████| 62 kB 1.7 MB/s 
     |████████████████████████████████| 168 kB 56.1 MB/s 
     |████████████████████████████████| 166 kB 71.1 MB/s 
     |████████████████████████████████| 166 kB 68.7 MB/s 
     |████████████████████████████████| 162 kB 65.6 MB/s 
     |████████████████████████████████| 162 kB 48.6 MB/s 
     |████████████████████████████████| 158 kB 70.0 MB/s 
     |████████████████████████████████| 157 kB 66.2 MB/s 
     |████████████████████████████████| 157 kB 65.2 MB/s 
     |████████████████████████████████| 157 kB 70.9 MB/s 
     |████████████████████████████████| 157 kB 59.9 MB/s 
     |████████████████████████████████| 157 kB 57.6 MB/s 
     |████████████████████████████

 # 1. Import

In [ ]:
import os
import json
import wandb
import shutil
import random
import evaluate
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm
from pathlib import Path
from omegaconf import OmegaConf
from sklearn.metrics import f1_score
from datetime import datetime, timezone, timedelta
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils import *
from torch.optim import Adam
from torch.utils.data import DataLoader

from transformers import logging, get_linear_schedule_with_warmup, get_cosine_with_hard_restarts_schedule_with_warmup, get_constant_schedule_with_warmup
from transformers import ( 
    BertConfig,
    ElectraConfig
)


# 실험에 사용할 모델 라이브러리 추가
from transformers import (
    BertTokenizer,
    BertTokenizerFast,
    AutoTokenizer,
    ElectraTokenizer,
    AlbertTokenizer,
    RobertaTokenizerFast
)
from transformers import (
    BertModel,
    AutoModel, 
    ElectraForSequenceClassification,
    BertForSequenceClassification,
    AlbertForSequenceClassification,
    RobertaForSequenceClassification,
    TextClassificationPipeline
)

# 2. 실험 세팅 
### Train Configuration

In [ ]:
# Project directory
PROJECT_DIR = os.getcwd()
print('PROJECT_DIR :',PROJECT_DIR)

# Load train config
## !!! config 수정할때마다 이 셀을 다시 실행하세요 !!!
config_path = os.path.join(PROJECT_DIR, 'baseline', 'config', 'train.yaml')
config = OmegaConf.load(config_path)

PROJECT_DIR : /content/drive/MyDrive/Shopping


### 실험 기록 및 경로 설정

In [ ]:
# Train Serial
kst = timezone(timedelta(hours=9))
train_serial = datetime.now(tz=kst).strftime("%Y%m%d_%H%M%S")


# Recorder directory
if config.train.resume==None:
    RECORDER_DIR = os.path.join(PROJECT_DIR, 'results', 'train', train_serial)
else:
    RECORDER_DIR = os.path.join(PROJECT_DIR, config.train.resume_weight_dir)
    
os.makedirs(RECORDER_DIR, exist_ok=True)
print("Results will be found here : ", RECORDER_DIR)

# Data Directory
DATA_DIR = Path(config.train.dataset_path)
TRAIN_DIR = DATA_DIR/'train/'
TEST_DIR = DATA_DIR/'test/'
SAMPLE_DIR = os.path.join(DATA_DIR,'sample_submission.csv')

print('DATA_DIR :',DATA_DIR)
print('TRAIN_DIR :',TRAIN_DIR)
print('TEST_DIR :',TEST_DIR)
print('SAMPLE_DIR :',SAMPLE_DIR)

Results will be found here :  /content/drive/MyDrive/Shopping/results/train/20221019_180402
DATA_DIR : /content/drive/MyDrive/Shopping/DATA
TRAIN_DIR : /content/drive/MyDrive/Shopping/DATA/train
TEST_DIR : /content/drive/MyDrive/Shopping/DATA/test
SAMPLE_DIR : /content/drive/MyDrive/Shopping/DATA/sample_submission.csv


### Seed 고정 및 GPU 지정

In [ ]:
# Seed 설정
random.seed(config.train.seed)
np.random.seed(config.train.seed)
torch.manual_seed(config.train.seed)
torch.cuda.manual_seed_all(config.train.seed)

torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

# GPU 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

### Set logger

In [ ]:
os.chdir('/content/drive/MyDrive/Shopping/baseline')
from modules.utils import get_logger

logger = get_logger(name='train', dir_=RECORDER_DIR, stream=False)
logger.info(f"Set Logger {RECORDER_DIR}")
logger.info(OmegaConf.to_yaml(config))

INFO:train:Set Logger /content/drive/MyDrive/Shopping/results/train/20221019_180402
INFO:train:train:
  dataset_path: /content/drive/MyDrive/Shopping/DATA
  num_classes: 14
  max_seq_len: 128
  train_batch_size: 32
  eval_batch_size: 32
  num_epochs: 1
  metric: f1
  seed: 777
  early_stopping_patience: 5
  resume: true
  resume_weight_dir: results/train/20221019_180402
  save_strategy: epoch
  save_steps: 100
  eval_steps: 500
  evaluation_strategy: epoch
  fp16: true
  num_workers: 1
  use_cuda: true
  gpus: '1'
  optimizer: adam
  adam_epsilon: 1.0e-08
  warmup_proportion: 0.04
  learning_rate: 5.0e-05
  weight_decay: 0.01
  wandb: true
  logging_strategy: epoch
  logging_steps: 200
model:
  pretrained_model: klue/roberta-base
  architecture: RobertaForSequenceClassification
  tokenizer_class: BertTokenizer



# 3. EDA 및 데이터 전처리
### DataFrame으로 시각화

In [ ]:
def make_df(data_dir:Path, test=False):
    sentence_id=[]     # 파일명 + .txt + 텍스트 내에서 문장 순서
    speaker = []       # 화자
    text = []          # text
    speechAct = []     # 문장이 어떤 내용인지 말해주는 label값
    
    for directory in data_dir.iterdir():
        for file in directory.iterdir():
            if 'checkpoint' in str(file): continue
            if test ==False:
                try:
                    label = json.load(open(file))
                except:
                    continue
                lines = label['info'][0]['annotations']['lines']
                for i, l in enumerate(lines):
                    sentence_id.append(file.stem+'.txt-'+str(i)) # 문장 id
                    speaker.append(l['text'].split('.')[0]) # 화자 넣기
                    text.append(l['text'].split('.')[-1]) # 화자 구분 빼기
                    speechAct.append(l['speechAct'])
            elif file.match("*.txt"):
                try:
                    with open(file, "r") as f:
                        lines = f.read().splitlines()
                except:
                    continue

                for i, l in enumerate(lines):
                    sentence_id.append(file.stem+'.txt-'+str(i))
                    speaker.append(l.split('.')[0]) # 화자 넣기
                    text.append(l.split('.')[-1]) # 화자 구분 빼기
    if test==False:
        df = pd.DataFrame({'sentence_id': sentence_id, 'speaker':speaker,'text': text, 'speechAct':speechAct})
    else:
        df = pd.DataFrame({'sentence_id': sentence_id, 'speaker':speaker,'text': text})

    return df

In [ ]:
train_df = make_df(TRAIN_DIR)

In [ ]:
train_df

,sentence_id,speaker,text,speechAct
0,shopping1_2424.txt-0,A,반갑습니다 상담사 #@이름#입니다,인사하기
1,shopping1_2424.txt-1,B,세탁기 산 지가 아직 일 년이 안 됐는데 고장이 났어요,진술하기
2,shopping1_2424.txt-2,A,고객님 세탁기의 종류는 무엇인가요,질문하기
3,shopping1_2424.txt-3,B,지금 일반 통돌이 쓰고 있어요,진술하기
4,shopping1_2424.txt-4,A,어떤 것이 고장이 났나요,질문하기
...,...,...,...,...
189778,shopping3_1565.txt-6,B,뭐 따로 해야 할 절차가 있을까요,질문하기
189779,shopping3_1565.txt-7,A,이 기능을 이용하기 위해서는요,진술하기
189780,shopping3_1565.txt-8,A,사용할 계좌 인증 및 자동이체 출금 동의 절차를 완료해야 합니다,진술하기
189781,shopping3_1565.txt-9,B,네 그렇군요 알겠습니다,진술하기


In [ ]:
# 라벨을 인코딩할 딕셔너리 (학습에 사용)
label2encoding = {Act: ind for ind, Act in enumerate(train_df['speechAct'].unique())}
# 인코딩된 라벨을 디코딩할 딕셔너리 (추론에 사용)
encoding2label = {idx: Act for idx, Act in enumerate(train_df['speechAct'].unique())}

In [ ]:
# 길이가 128이 넘는 코멘트가 있는지 확인
train_df['text'][train_df['text'].str.len()>config.train.max_seq_len]

Series([], Name: text, dtype: object)

### Tokenizer

In [ ]:
# config.json 에서 지정 이름별로 가져올 라이브러리 지정

TOKENIZER_CLASSES = {
    "BertTokenizer": BertTokenizer,
    "BertTokenizerFast": BertTokenizerFast,
    "AutoTokenizer": AutoTokenizer,
    "ElectraTokenizer": ElectraTokenizer,
    "AlbertTokenizer": AlbertTokenizer,
    "RobertaTokenizerFast" : RobertaTokenizerFast
}
TOKENIZER = TOKENIZER_CLASSES[config.model.tokenizer_class].from_pretrained(config.model.pretrained_model)

Downloading:   0%|          | 0.00/248k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/173 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/375 [00:00<?, ?B/s]

In [ ]:
# Tokenizer 예시
comment_ex = train_df['text'][0]
print(TOKENIZER(comment_ex))
print(TOKENIZER.encode(comment_ex),"\n")

# 토큰으로 나누기
print(TOKENIZER.tokenize(comment_ex),"\n")

# 토큰 id로 매핑하기
print(TOKENIZER.convert_tokens_to_ids(TOKENIZER.tokenize(comment_ex)))

{'input_ids': [0, 9927, 2219, 3606, 4981, 2063, 7, 36, 3934, 7, 3714, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
[0, 9927, 2219, 3606, 4981, 2063, 7, 36, 3934, 7, 3714, 2] 

['반갑', '##습', '##니다', '상담', '##사', '#', '@', '이름', '#', '입니다'] 

[9927, 2219, 3606, 4981, 2063, 7, 36, 3934, 7, 3714]


# 4. Dataset

### CustomDataset 클래스 정의

In [ ]:
class CustomDataset(torch.utils.data.Dataset):  # 데이터를 input으로 변환해주는 Dataset 클래스를 상속한 후 커스터마이징

    def __init__(self, df, tokenizer, max_seq_len, mode = 'train'):  # Dataset 클래스는 기본적으로 __init__, __len__, __getitem__를 정의해 주어야 한다

        self.data = df
        self.tokenizer = tokenizer
        self.max_seq_len = max_seq_len
        self.mode = mode
        self.speakers = self.data['speaker'].tolist()
        self.texts = self.data['text'].tolist()
        
        if self.mode!='test':
            try: 
                self.labels = df['speechAct'].tolist()
            except:
                assert False, 'CustomDataset Error : \'label\' column does not exist in the dataframe'
     
    def __len__(self):  # index를 통해 input을 순차적으로 읽어오기 위해서는 데이터의 길이가 먼저 확인되어야 한다. __len__ 함수는 input의 길이를 반환해주는 함수
        return len(self.data)
                

    def __getitem__(self, idx):  # input의 길이가 확인되면 index를 통해 데이터를 불러올 수 있다. __getitem__ 함수는 index에 해당하는 input 데이터를 반환해주는 함수
        """
        전체 데이터에서 특정 인덱스 (idx)에 해당하는 기사제목과 댓글 내용을 
        토크나이즈한 data('input_ids', 'attention_mask','token_type_ids')의 딕셔너리 형태로 불러옴
        """
        tokenized_text = self.tokenizer(self.texts[idx],
                             padding= 'max_length',
                             max_length=self.max_seq_len,
                             truncation=True,
                             return_token_type_ids=True,
                             return_attention_mask=True,
                             return_tensors = "pt")

        if self.mode=='test':
            data = {'input_ids': tokenized_text['input_ids'].squeeze(0).long(),
                   'attention_mask': tokenized_text['attention_mask'].squeeze(0).long(),
                   'token_type_ids': tokenized_text['token_type_ids'].squeeze(0).long()
                    }
        else:
            data = {'input_ids': tokenized_text['input_ids'].squeeze(0).long(),
                   'attention_mask': tokenized_text['attention_mask'].squeeze(0).long(),
                   'token_type_ids': tokenized_text['token_type_ids'].squeeze(0).long(),
                    'labels': label2encoding[self.labels[idx]]
                    }


        return data

### Train, Validation set 나누기

In [ ]:
train_data, val_data = train_test_split(train_df, test_size=0.2, random_state=config.train.seed)

train_dataset = CustomDataset(train_data, TOKENIZER, config.train.max_seq_len, 'train')
val_dataset = CustomDataset(val_data, TOKENIZER, config.train.max_seq_len, 'validation')

print("Train dataset: ", len(train_dataset))
print("Validation dataset: ", len(val_dataset))

Train dataset:  151826
Validation dataset:  37957


### DataLoader

In [ ]:
# torch.utils.data.DataLoader : input을 배치 단위로 리턴해주는 기능

train_dataloader = DataLoader(dataset=train_dataset,
                                batch_size=config.train.train_batch_size,
                                num_workers=config.train.num_workers, 
                                shuffle=True,
                                pin_memory=True,
                                drop_last=False)

val_dataloader = DataLoader(dataset=val_dataset,
                            batch_size=config.train.eval_batch_size,
                            num_workers=config.train.num_workers, 
                            shuffle=False,
                            pin_memory=True,
                            drop_last=False)


# 5. 모델 선언

In [ ]:
from transformers import logging
logging.set_verbosity_error()

# config.json 에 입력된 architecture 에 따라 베이스 모델 설정
BASE_MODELS = {
    "BertForSequenceClassification": BertForSequenceClassification,
    "AutoModel": AutoModel,
    "ElectraForSequenceClassification": ElectraForSequenceClassification,
    "AlbertForSequenceClassification": AlbertForSequenceClassification,
    "RobertaForSequenceClassification": RobertaForSequenceClassification
}

model = BASE_MODELS[config.model.architecture].from_pretrained(config.model.pretrained_model, num_labels = config.train.num_classes)
# print(model)

Downloading:   0%|          | 0.00/546 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/443M [00:00<?, ?B/s]

In [ ]:
# # 학습을 재개할 시 config 파일의 resume_weight_dir를 설정해주세요
# if config.train.resume :
#     checkpoint = torch.load(config.train.resume_weight_dir)
#     model.load_state_dict(checkpoint['model'])

# 6. Optimizer

In [ ]:
optimizer = Adam(model.parameters(), lr=config.train.learning_rate, eps=config.train.adam_epsilon)

total_steps = len(train_dataloader) * config.train.num_epochs 
lr_scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=int(total_steps * config.train.warmup_proportion), 
                                                num_training_steps=total_steps)

# lr_scheduler_test
# lr_scheduler = get_cosine_with_hard_restarts_schedule_with_warmup(optimizer, num_warmup_steps=int(total_steps * config.train.warmup_proportion), num_training_steps=total_steps)
# lr_scheduler = get_constant_schedule_with_warmup(optimizer, num_warmup_steps=int(total_steps * config.train.warmup_proportion))


# 7. Metric

- [evaluate 라이브러리 document](https://pypi.org/project/evaluate/)
- [각 metric 사용법](https://huggingface.co/evaluate-metric?sort_spaces=alphabetical#spaces)

In [ ]:
# 사용 가능한 metric 리스트를 보려면 아래를 주석 해제하여 확인하여 config 파일을 수정하여 다시 불러오세요.
# evaluate.list_evaluation_modules()

def compute_metrics(eval_pred):
    metric_fn = evaluate.load(config.train.metric)
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    if config.train.metric=='f1':
        score = metric_fn.compute(predictions=predictions, references=labels, average='macro')
    else:
        score = metric_fn.compute(predictions=predictions, references=labels)
    return score

# 8. Early stopper

In [ ]:
from modules.earlystoppers import EarlyStopper

early_stopper = EarlyStopper(patience=config.train.early_stopping_patience,
                             mode='min',
                             logger=logger)

INFO:train:Initiated ealry stopper, mode: min, best score: inf, patience: 5


# 9. Recorder

In [ ]:
from modules.recorders import Recorder

recorder = Recorder(record_dir=RECORDER_DIR,
                    model=model,
                    optimizer=optimizer,
                    scheduler=None,
                    amp=None,
                    logger=logger)

In [ ]:
# Wandb
# Wandb 사용법은 11월 21일 곽대훈 멘토님의 강의를 참고하세요!
if config.train.wandb:
    
    wandb_project_serial = 'shopping_conversation'
    os.environ["WANDB_NOTEBOOK_NAME"] = wandb_project_serial
    wandb_username = 'cheche7' # 각자의 계정으로 수정하세요.
    wandb.init(project=wandb_project_serial, dir=RECORDER_DIR, entity=wandb_username)
    wandb.run.name = train_serial
    wandb.config.update(config)
    wandb.watch(model)


wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find shopping_conversation.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


# 10. Trainer Class 정의

### 학습 파라미터 설정

- [transformers Trainer 설명](https://huggingface.co/docs/transformers/main_classes/trainer)
- [transformers TrainingArguments 파라미터 설명](https://huggingface.co/docs/transformers/v4.23.1/en/main_classes/trainer#transformers.TrainingArguments) : Training Arguments 파라미터 설명이 기술되어 있으니 참고하시어 각 파라미터가 의미하는 바를 알아보고 조정, 혹은 추가해보세요.

In [ ]:
# transformers에서 Trainer를 불러와서 학습에 사용할텐데, 그때 사용할 파라미터들을 미리 지정해둔다

from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir= RECORDER_DIR,  # output directory
    num_train_epochs=config.train.num_epochs,  # total number of training epochs
    resume_from_checkpoint = config.train.resume_weight_dir,
    per_device_train_batch_size=config.train.train_batch_size,  # batch size per device during training
    per_device_eval_batch_size=config.train.eval_batch_size,  # batch size for evaluation
    learning_rate = config.train.learning_rate,
    warmup_ratio = config.train.warmup_proportion,
    weight_decay=config.train.weight_decay,
    logging_strategy = config.train.logging_strategy,
    # logging_steps=config.train.logging_steps, # valid when logging_strategy ='steps'
    save_strategy=config.train.save_strategy,
    # save_steps=config.train.save_steps, # valid when save_strategy ='steps'
    # eval_steps=config.train.eval_steps, # valid when save_strategy ='steps'
    do_train=True, # Perform training
    do_eval=True, # Perform evaluation
    evaluation_strategy = config.train.evaluation_strategy, # evalute after each epoch
    gradient_accumulation_steps = 64,  # total number of steps before back propagation
    fp16 = config.train.fp16, # Use mixed precision
    run_name = train_serial, # experiment name, typically used for wandb
    report_to="wandb",  # disable wandb
    load_best_model_at_end = True,
    overwrite_output_dir=True,
    save_total_limit=3,
    seed= config.train.seed  # Seed for experiment reproducibility
)

## 학습 

In [ ]:
class Run:

    def __init__(self, trainer, tokenizer, training_args, test=None, submission_name = None):
        self.trainer = trainer
        self.tokenizer = tokenizer
        self.training_args = training_args
        self.test = test
        self.submission_name = submission_name

    def __call__(self):
        if self.training_args.do_train:
            self.train()

        if self.training_args.do_eval:
            self.validate()

        if self.training_args.do_predict and self.test is not None:
            self.predict()

    def train(self):
        self.trainer.train()
        self.trainer.save_model()
        if self.trainer.is_world_process_zero():
            self.tokenizer.save_pretrained(RECORDER_DIR)

    def validate(self):
        logger.info("*** Evaluate ***")
        result = self.trainer.evaluate()
        output_eval_file = os.path.join(RECORDER_DIR, "eval_results.txt")
        if self.trainer.is_world_process_zero():
            with open(output_eval_file, "w") as writer:
                logger.info("***** Eval results *****")
                for key, value in result.items():
                    logger.info("%s = %s", key, value)
                    writer.write("%s = %s\n" % (key, value))

        logger.info("Validation set result : {}".format(result))

    # 추론 함수
    def predict(self):
        logger.info("*** Test ***")
        predictions = self.trainer.predict(test_dataset=self.test)
        test_df['speechAct'] = self.prediction(predictions.predictions)
        test_df.to_csv(self.submission_name, index=False)
        test_df.sort_values(by='sentence_id', ascending=True)
        print("Submission file saved as :", self.submission_name)

    def prediction(self,logit):
        labels = np.argmax(logit, axis=1)
        return list(map(lambda x: encoding2label[x], labels))

# 11. Trainer 선언

In [ ]:
# Trainer
from transformers import Trainer
from transformers import EarlyStoppingCallback

# Set trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
callbacks=[EarlyStoppingCallback(early_stopping_patience=config.train.early_stopping_patience)]

)

# Set runner
train = Run(
    training_args=training_args,
    trainer=trainer,
    tokenizer=TOKENIZER,
    )

Using cuda_amp half precision backend


# 12. 학습

In [ ]:
train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 151826
  Num Epochs = 1
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 2048
  Gradient Accumulation steps = 64
  Total optimization steps = 74
  Number of trainable parameters = 110628878
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
***** Running Evaluation *****
  Num examples = 37957
  Batch size = 32


{'loss': 0.9466, 'learning_rate': 0.0, 'epoch': 1.0}


Saving model checkpoint to /content/drive/MyDrive/Shopping/results/train/20221019_180402/checkpoint-74


{'eval_loss': 0.6119049191474915, 'eval_f1': 0.4359302576770813, 'eval_runtime': 82.389, 'eval_samples_per_second': 460.705, 'eval_steps_per_second': 14.407, 'epoch': 1.0}


Configuration saved in /content/drive/MyDrive/Shopping/results/train/20221019_180402/checkpoint-74/config.json
Model weights saved in /content/drive/MyDrive/Shopping/results/train/20221019_180402/checkpoint-74/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from /content/drive/MyDrive/Shopping/results/train/20221019_180402/checkpoint-74 (score: 0.6119049191474915).
Saving model checkpoint to /content/drive/MyDrive/Shopping/results/train/20221019_180402


{'train_runtime': 1034.7582, 'train_samples_per_second': 146.726, 'train_steps_per_second': 0.072, 'train_loss': 0.9465766081938872, 'epoch': 1.0}


Configuration saved in /content/drive/MyDrive/Shopping/results/train/20221019_180402/config.json
Model weights saved in /content/drive/MyDrive/Shopping/results/train/20221019_180402/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/Shopping/results/train/20221019_180402/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/Shopping/results/train/20221019_180402/special_tokens_map.json
INFO:train:*** Evaluate ***
***** Running Evaluation *****
  Num examples = 37957
  Batch size = 32


{'eval_loss': 0.6119049191474915, 'eval_f1': 0.4359302576770813, 'eval_runtime': 82.3924, 'eval_samples_per_second': 460.686, 'eval_steps_per_second': 14.407, 'epoch': 1.0}


INFO:train:***** Eval results *****
INFO:train:eval_loss = 0.6119049191474915
INFO:train:eval_f1 = 0.4359302576770813
INFO:train:eval_runtime = 82.3924
INFO:train:eval_samples_per_second = 460.686
INFO:train:eval_steps_per_second = 14.407
INFO:train:epoch = 1.0
INFO:train:Validation set result : {'eval_loss': 0.6119049191474915, 'eval_f1': 0.4359302576770813, 'eval_runtime': 82.3924, 'eval_samples_per_second': 460.686, 'eval_steps_per_second': 14.407, 'epoch': 1.0}


# 13. 추론 
### 추론 config 설정

In [ ]:
# Load config
predict_config_path = os.path.join(PROJECT_DIR, 'baseline', 'config', 'predict.yaml')
predict_config = OmegaConf.load(predict_config_path)

### 추론을 로깅할 디렉토리 생성

In [ ]:
# Serial
if predict_config['predict']['train_serial'] is None:
    train_serial = os.path.basename(RECORDER_DIR)
else:
    train_serial = predict_config['predict']['train_serial']
    

# Predict directory
PREDICT_DIR = os.path.join(RECORDER_DIR, 'predict')
os.makedirs(PREDICT_DIR, exist_ok=True)

### 데이터 경로 지정

In [ ]:
# Data Directory
DATA_DIR = predict_config['predict']['dataset_path']

### 테스트 데이터셋 및 데이터 로더 선언


In [ ]:
test_df = make_df(TEST_DIR, test=True)
test_dataset = CustomDataset(test_df, TOKENIZER, config.train.max_seq_len, 'test')
print("Test dataset: ", len(test_dataset))

test_dataloader = DataLoader(dataset=test_dataset,
                            batch_size=predict_config.predict.batch_size,
                            num_workers=predict_config.predict.num_workers,
                            shuffle=False,
                            pin_memory=True,
                            drop_last=False)

Test dataset:  20520


/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
test_df

,sentence_id,speaker,text
0,test_0901.txt-0,A,반갑습니다 즐거운 쇼핑 #@소속#입니다
1,test_0901.txt-1,A,어떤 문의가 있으신가요
2,test_0901.txt-2,B,네 궁금한 게 있어요
3,test_0901.txt-3,B,쿠폰과 이벤트 코드는 뭐가 다르죠
4,test_0901.txt-4,A,이벤트 코드는 다양한 수단으로 고객님께 전달됩니다
...,...,...,...
20515,test_0980.txt-12,A,아무래도 그런 걸 구입하시는 게 금액이 더 저렴합니다
20516,test_0980.txt-13,B,그럼 마트가서 사야겠네요
20517,test_0980.txt-14,A,네 저도 같은 제품 사용하고 있는데 다른 제품 사서 쓰고 있거든요
20518,test_0980.txt-15,B,그러세요 저도 그렇게 해야겠네요 감사합니다


In [ ]:
model = BASE_MODELS[config.model.architecture].from_pretrained(RECORDER_DIR)

predicting_args = TrainingArguments(
        run_name=train_serial,
        disable_tqdm=False,
        per_device_eval_batch_size = predict_config.predict.batch_size,
        fp16=config.train.fp16,
        gradient_accumulation_steps=64,
        do_train=False,
        do_eval=False,
        do_predict=True,
        output_dir='.',
    )

trainer_prediction = Trainer(
    model= model,
    args=predicting_args
)

predict = Run(
    training_args=predicting_args,
    trainer=trainer_prediction,
    tokenizer=TOKENIZER,
    test=test_dataset,
    submission_name = predict_config.predict.submission_name
    )

loading configuration file /content/drive/MyDrive/Shopping/results/train/20221019_180402/config.json
Model config RobertaConfig {
  "_name_or_path": "klue/roberta-base",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_10": 10,
    "LABEL_11": 11,
    "LABEL_12": 12,
    "LABEL_13": 13,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABE

## 추론

In [ ]:
predict()

INFO:train:*** Test ***
***** Running Prediction *****
  Num examples = 20520
  Batch size = 1


Submission file saved as : submission.csv
